In [1]:
import pickle

In [2]:
file_names = ['url_list_women.p', 'url_list_women_sale.p', 
              'url_list_men.p', 'url_list_men_sale.p']

In [3]:
item_url_list = []

In [4]:
for file_name in file_names:
    f = open('pickle_data/' + file_name, 'rb')
    item_url_list = item_url_list + pickle.load(f)
    f.close()

In [5]:
len(item_url_list)

8528

In [6]:
item_url_list[:10]

['http://www.hm.com/us/product/65274?article=65274-B',
 'http://www.hm.com/us/product/65274?article=65274-E',
 'http://www.hm.com/us/product/67562?article=67562-D',
 'http://www.hm.com/us/product/54630?article=54630-F',
 'http://www.hm.com/us/product/66996?article=66996-B',
 'http://www.hm.com/us/product/22590?article=22590-V',
 'http://www.hm.com/us/product/67571?article=67571-A',
 'http://www.hm.com/us/product/70360?article=70360-A',
 'http://www.hm.com/us/product/70480?article=70480-C',
 'http://www.hm.com/us/product/64155?article=64155-B']

In [7]:
import urllib
def return_item_page(url_item):
    handle = urllib.urlopen(url_item)
    html_gunk =  handle.read()
    handle.close()
    return str(html_gunk)

In [8]:
def get_metricCategoryID(item_page):
    idx_start = item_page.find("metricCategoryID") + len('metricCategoryID":"')
    idx_end = idx_start + item_page[idx_start:].find('"')
    
    return item_page[idx_start:idx_end]

def check_metricCategoryID(metricCategoryID):
    category_list = ['TOPS', 'BOTTOMS', 'SHIRT', 'VESTS' 'BLOUSES', 'DRESSES', 'JUMPSUITS', 'JEANS', 'PANTS', 'TROUSERS', 'CARDIGANS', 'SWEATERS', 'JUMPERS', 'HOODIES', 
               'SWEATSHIRTS', 'SHORTS', 'SKIRTS', 'JACKETS', 'COATS', 'BLAZERS', 'SUITS']
    for category in category_list:
        if metricCategoryID.find(category) != -1:
            return True
    return False

def check_category(item_page):
    return check_metricCategoryID(get_metricCategoryID(item_page))

In [39]:
def get_info(url_item):
    # url_item = "http://www.hm.com/us/product/72818?article=72818-A"
    item_page = return_item_page(url_item)
    parser = HnM_ProductPage_Parser()
    parser.feed(item_page)

    item_serial = url_item[url_item.find("article=")+len("article="):]

    return {'serial':item_serial, 
            'name':parser.get_name(), 
            'color':parser.get_color(), 
            'who':parser.get_who(), 
            'image': parser.get_image_urls(), 
            #'title':parser.get_title(), 
            #'title breaks':parser.get_title_breaks(), 
            'url':url_item, 
            'metricCategoryID':get_metricCategoryID(item_page)}

In [10]:
import urllib
from HTMLParser import HTMLParser

class HnM_ProductPage_Parser(HTMLParser):
    in_head_tag = False
    in_title_tag = False
    image_urls = dict()
    name = ""
    color = ""
    who = ""
    title = ""
    title_breaks = 0
    
    def handle_starttag(self, tag, attrs):
        if tag == "head":
            self.in_head_tag = True
        if tag == "title":
            self.in_title_tag = True
        if tag == "a":
            for attr in attrs:
                if attr[0] == "class" and (attr[1] == "FASHION_FRONT" or attr[1] == "STUDIO_FRONT"):
                    for attr2 in attrs:
                        if attr2[0] == "href":
                            self.image_urls["model"] = "http:" + attr2[1]
                
                if attr[0] == "class" and attr[1] == "STILL_LIFE_FRONT":
                    for attr2 in attrs:
                        if attr2[0] == "href":
                            self.image_urls["item"] = "http:" + attr2[1]
        
    def handle_endtag(self, tag):
        if tag == "head":
            self.in_head_tag = False
        if tag == "title":
            self.in_title_tag = False
            data_list = self.title.split(' | ')
            self.name =  data_list[0]
            self.color = data_list[1]
            self.who = data_list[2]
            
    def handle_data(self, data):
        if self.in_head_tag and self.in_title_tag: # handling an unknown error 
            if self.title == "":
                self.title = self.title + data
            else:
                self.title = self.title + '&M' + data
            self.title_breaks = self.title_breaks + 1
        
    def get_image_urls(self):
        return self.image_urls
    
    def get_name(self):
        return self.name
    
    def get_color(self):
        return self.color
    
    def get_who(self):
        return self.who
    
    def get_title(self):
        return self.title
    
    def get_title_breaks(self):
        return self.title_breaks

### Get the information of an item

In [40]:
item_info = get_info('http://www.hm.com/us/product/65274?article=65274-B')

In [24]:
item_info

{'color': 'White',
 'image': {'item': 'http://lp.hm.com/hmprod?set=key[source],value[/model/2017/E00 0478110 001 77 1688.jpg]&set=key[rotate],value[]&set=key[width],value[]&set=key[height],value[]&set=key[x],value[]&set=key[y],value[]&set=key[type],value[STILL_LIFE_FRONT]&set=key[hmver],value[2]&call=url[file:/product/large]',
  'model': 'http://lp.hm.com/hmprod?set=key[source],value[/environment/2017/8GZ_0158_011R.jpg]&set=key[rotate],value[0]&set=key[width],value[3927]&set=key[height],value[4591]&set=key[x],value[0]&set=key[y],value[-1]&set=key[type],value[FASHION_FRONT]&set=key[hmver],value[0]&call=url[file:/product/large]'},
 'name': 'Short-sleeved Top',
 'serial': '65274-B',
 'url': 'http://www.hm.com/us/product/65274?article=65274-B',
 'who': 'Women'}

In [27]:
item_info['serial']

'65274-B'

In [26]:
item_info['name']

'Short-sleeved Top'

In [28]:
item_info['color']

'White'

In [29]:
item_info['who']

'Women'

In [32]:
print item_info['image']['model']

http://lp.hm.com/hmprod?set=key[source],value[/environment/2017/8GZ_0158_011R.jpg]&set=key[rotate],value[0]&set=key[width],value[3927]&set=key[height],value[4591]&set=key[x],value[0]&set=key[y],value[-1]&set=key[type],value[FASHION_FRONT]&set=key[hmver],value[0]&call=url[file:/product/large]


In [33]:
print item_info['image']['model']

http://lp.hm.com/hmprod?set=key[source],value[/environment/2017/8GZ_0158_011R.jpg]&set=key[rotate],value[0]&set=key[width],value[3927]&set=key[height],value[4591]&set=key[x],value[0]&set=key[y],value[-1]&set=key[type],value[FASHION_FRONT]&set=key[hmver],value[0]&call=url[file:/product/large]


In [41]:
item_info['metricCategoryID']

'LADIES_TOPS_SHORT_SLEEVED_W112'

### Get the information of all items

### Count title breaks probably by '&M'

In [10]:
count_breaks_dict = dict()
count_loop = 0
for item_url_list in item_url_lists[:1]:
    for item_url in item_url_list[700:900]:
        item_info = get_info(item_url)
        if item_info['title breaks'] > 2:
            print "[" + str(item_info['title breaks']) + "] " + item_info['title']
            print item_info['url']
            if item_info['title breaks'] in count_breaks_dict:
                count_breaks_dict[item_info['title breaks']] = count_breaks_dict[item_info['title breaks']] + 1
            else:
                count_breaks_dict[item_info['title breaks']] = 1
        if count_loop % 100 == 0:
            print "count loops:" , count_loop, "===================================="
        count_loop = count_loop + 1
print "total count loops:" , count_loop, "===================================="
for breaks, count_breaks in count_breaks_dict.items():
    print breaks, "breaks count:", count_breaks

count loops: 0 ====================================
[3] H+ Printed T-shirt | Powder pink | Women | H US
http://www.hm.com/us/product/70852?article=70852-A
count loops: 100 ====================================
[3] H+ Denim Shorts | Denim blue | Women | H US
http://www.hm.com/us/product/45083?article=45083-C
[3] H+ Denim Shorts | Black denim | Women | H US
http://www.hm.com/us/product/45083?article=45083-A
total count loops: 200 ====================================
3 breaks count: 3


In [11]:
count_breaks_dict = dict()
count_loop = 0
for item_url_list in item_url_lists:
    for item_url in item_url_list:
        item_info = get_info(item_url)
        if item_info['title breaks'] > 2:
            print "[" + str(item_info['title breaks']) + "] " + item_info['title']
            print item_info['url']
            if item_info['title breaks'] in count_breaks_dict:
                count_breaks_dict[item_info['title breaks']] = count_breaks_dict[item_info['title breaks']] + 1
            else:
                count_breaks_dict[item_info['title breaks']] = 1
        if count_loop % 100 == 0:
            print "count loops:" , count_loop, "===================================="
        count_loop = count_loop + 1
print "total count loops:" , count_loop, "===================================="
for breaks, count_breaks in count_breaks_dict.items():
    print breaks, "breaks count:", count_breaks

count loops: 0 ====================================
count loops: 100 ====================================
count loops: 200 ====================================
count loops: 300 ====================================
count loops: 400 ====================================
count loops: 500 ====================================
count loops: 600 ====================================
count loops: 700 ====================================
[3] H+ Printed T-shirt | Powder pink | Women | H US
http://www.hm.com/us/product/70852?article=70852-A
count loops: 800 ====================================
[3] H+ Denim Shorts | Denim blue | Women | H US
http://www.hm.com/us/product/45083?article=45083-C
[3] H+ Denim Shorts | Black denim | Women | H US
http://www.hm.com/us/product/45083?article=45083-A
count loops: 900 ====================================
[3] H+ Jersey Tunic | Black | Women | H US
http://www.hm.com/us/product/69089?article=69089-A
[3] H+ V-neck Linen Top | Gray melange | Women | H US
http://www.h

[3] H+ City Shorts | Dark blue | Women | H US
http://www.hm.com/us/product/68552?article=68552-A
count loops: 2700 ====================================
[3] H+ Draped Tankini Top | Pink/paisley | Women | H US
http://www.hm.com/us/product/65541?article=65541-A
[3] H+ Denim Skirt | Light denim blue | Women | H US
http://www.hm.com/us/product/66630?article=66630-A
[3] H+ Crushed-velvet Shorts | Light heather | Women | H US
http://www.hm.com/us/product/69590?article=69590-A
[3] H+ Linen-blend Pants | Black | Women | H US
http://www.hm.com/us/product/64767?article=64767-A
[3] H+ Straight Regular Jeans | Dark denim blue | Women | H US
http://www.hm.com/us/product/65485?article=65485-A
count loops: 2800 ====================================
[3] H+ Shaping Skinny High waist | Denim blue | Women | H US
http://www.hm.com/us/product/67455?article=67455-A
[3] H+ Slim Regular Ankle Jeans | Denim blue | Women | H US
http://www.hm.com/us/product/67485?article=67485-B
count loops: 2900 =================

[3] H+ Bikini Bottoms | Powder pink | Women | H US
http://www.hm.com/us/product/63897?article=63897-B
[3] H+ Lyocell Shirt | Denim blue | Women | H US
http://www.hm.com/us/product/62210?article=62210-A
count loops: 4000 ====================================
[3] H+ Linen-blend Shorts | Natural white | Women | H US
http://www.hm.com/us/product/68085?article=68085-A
[3] H+ Lace Dress | Powder pink | Women | H US
http://www.hm.com/us/product/68620?article=68620-B
[3] H+ Hooded Sweatshirt Jacket | Powder | Women | H US
http://www.hm.com/us/product/67500?article=67500-A
[3] H+ Bikini Bottoms | Teal | Women | H US
http://www.hm.com/us/product/63908?article=63908-A
[3] H+ Short Dress | Black | Women | H US
http://www.hm.com/us/product/65571?article=65571-A
[3] H+ Crêpe Jacket | Black | Women | H US
http://www.hm.com/us/product/63385?article=63385-A
count loops: 4100 ====================================
[3] H+ Linen Shirt | Red/white stripe | Women | H US
http://www.hm.com/us/product/65772?artic

[3] H+ Sweatshirt | Pink/birds | Women | H US
http://www.hm.com/us/product/62518?article=62518-A
count loops: 5700 ====================================
[3] H+ One-shoulder Top | Black | Women | H US
http://www.hm.com/us/product/59360?article=59360-A
[3] H+ Rib-knit Cardigan | Black | Women | H US
http://www.hm.com/us/product/59420?article=59420-A
[3] H+ Long-sleeved Top | Black | Women | H US
http://www.hm.com/us/product/63396?article=63396-A
[3] H+ Wrap-front Skirt | Black | Women | H US
http://www.hm.com/us/product/52485?article=52485-B
[3] H+ Bikini Bottoms | Black | Women | H US
http://www.hm.com/us/product/43499?article=43499-B
[3] H+ Slim High Ankle Jeans | Denim blue/washed | Women | H US
http://www.hm.com/us/product/65705?article=65705-A
[3] H+ Knit Tunic | Light gray melange | Women | H US
http://www.hm.com/us/product/60300?article=60300-A
[3] H+ Pleated Camisole Top | Black | Women | H US
http://www.hm.com/us/product/52941?article=52941-A
[3] H+ Short Dress | Dark gray | Wome

count loops: 10600 ====================================
count loops: 10700 ====================================
count loops: 10800 ====================================
count loops: 10900 ====================================
count loops: 11000 ====================================
count loops: 11100 ====================================
count loops: 11200 ====================================
count loops: 11300 ====================================
count loops: 11400 ====================================
count loops: 11500 ====================================
count loops: 11600 ====================================
count loops: 11700 ====================================
total count loops: 11739 ====================================
3 breaks count: 268


In [13]:
count_items = 0
for item_url_list in item_url_lists:
    count_items = count_items + len(item_url_list)
print count_items

11739


* All 11739 item pages have been successfully parsed.
* Only '&M' was omitted. Fix HnM_ProductPage_Parser by adding '&M' to the right place.
 * Case 1: 'H&M+'
 * Case 2: 'H&M'

__Test__

In [17]:
count_breaks_dict = dict()
count_loop = 0
for item_url_list in item_url_lists[:1]:
    for item_url in item_url_list[700:900]:
        item_info = get_info(item_url)
        if item_info['title breaks'] > 2:
            print "[" + str(item_info['title breaks']) + "] " + item_info['title']
            print item_info['url']
            if item_info['title breaks'] in count_breaks_dict:
                count_breaks_dict[item_info['title breaks']] = count_breaks_dict[item_info['title breaks']] + 1
            else:
                count_breaks_dict[item_info['title breaks']] = 1
        if count_loop % 100 == 0:
            print "count loops:" , count_loop, "===================================="
        count_loop = count_loop + 1
print "total count loops:" , count_loop, "===================================="
for breaks, count_breaks in count_breaks_dict.items():
    print breaks, "breaks count:", count_breaks

count loops: 0 ====================================
[3] H&M+ Printed T-shirt | Powder pink | Women | H&M US
http://www.hm.com/us/product/70852?article=70852-A
count loops: 100 ====================================
[3] H&M+ Denim Shorts | Denim blue | Women | H&M US
http://www.hm.com/us/product/45083?article=45083-C
[3] H&M+ Denim Shorts | Black denim | Women | H&M US
http://www.hm.com/us/product/45083?article=45083-A
total count loops: 200 ====================================
3 breaks count: 3


__The problem has been solved!__